In [1]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.window import Window
from pyspark import SparkConf

In [2]:
conf = SparkConf()
spark_context = SparkSession.builder.config(conf=conf).getOrCreate()

26/02/02 22:13:49 WARN Utils: Your hostname, MacBook-Air-Kateryna.local resolves to a loopback address: 127.0.0.1; using 192.168.0.242 instead (on interface en0)
26/02/02 22:13:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 22:13:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", 
                                                       "org.apache.hadoop:hadoop-aws-2.7.3").appName("spark-hw").getOrCreate()

26/02/02 22:13:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
print(spark.version)

3.5.1


In [5]:
actor_df = spark.read.csv('../data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('../data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('../data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('../data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('../data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('../data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('../data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('../data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('../data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('../data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('../data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('../data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('../data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('../data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('../data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [6]:
film_category_group_df = (film_category_df.groupBy("category_id")
                                         .agg(F.count("film_id").alias("films_count"))
                                         .join(category_df.select("category_id", "name"), on="category_id", how="inner")
                                         .select("name", "films_count")
                                         .orderBy(F.col("films_count").desc())
                         )

film_category_group_df.show()

+-----------+-----------+
|       name|films_count|
+-----------+-----------+
|     Sports|         74|
|    Foreign|         73|
|     Family|         69|
|Documentary|         68|
|  Animation|         66|
|     Action|         64|
|        New|         63|
|      Drama|         62|
|      Games|         61|
|     Sci-Fi|         61|
|   Children|         60|
|     Comedy|         58|
|     Travel|         57|
|   Classics|         57|
|     Horror|         56|
|      Music|         51|
+-----------+-----------+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [7]:
actor_group_df = (rental_df.join(inventory_df.select("inventory_id", "film_id"), on="inventory_id", how="inner")
                           .join(film_actor_df.select("film_id", "actor_id"), on="film_id", how="inner") 
                           .join(actor_df.select("actor_id", "first_name", "last_name"), on="actor_id", how="inner") 
                           .groupBy("actor_id", "first_name", "last_name")
                           .agg(F.count("rental_id").alias("rental_count"))
                           .select(F.concat_ws(" ", "first_name", "last_name").alias("actor_name"),
                                   "rental_count")
                           .orderBy(F.col("rental_count").desc())
                           .limit(10)
                 )

actor_group_df.show()

+------------------+------------+
|        actor_name|rental_count|
+------------------+------------+
|    GINA DEGENERES|         753|
|    MATTHEW CARREY|         678|
|       MARY KEITEL|         674|
|ANGELA WITHERSPOON|         654|
|       WALTER TORN|         640|
|       HENRY BERRY|         612|
|       JAYNE NOLTE|         611|
|        VAL BOLGER|         605|
|     SANDRA KILMER|         604|
|      SEAN GUINESS|         599|
+------------------+------------+



3.
Вивести категорію фільмів, на яку було витрачено найбільше грошей
в прокаті

In [8]:
category_group_df = (rental_df.join(payment_df.select("rental_id", "amount"), on="rental_id", how="inner")
                              .join(inventory_df.select("inventory_id", "film_id"), on="inventory_id", how="inner") 
                              .join(film_category_df.select("film_id", "category_id"), on="film_id", how="inner") 
                              .join(category_df.select("category_id", "name"), on="category_id", how="inner") 
                              .groupBy("name")
                              .agg(F.sum("amount").alias("total_amt"))
                              .orderBy(F.col("total_amt").desc())
                              .limit(1)
                    )

category_group_df.show()

+------+-----------------+
|  name|        total_amt|
+------+-----------------+
|Sports|5314.209999999848|
+------+-----------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [9]:
(film_df.join(inventory_df.select("film_id", "inventory_id"), on="film_id", how="left")
        .filter(F.col("inventory_id").isNull())
        .select("title")
        .show()
)

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [10]:
children_category_df = (film_df.join(film_category_df.select("film_id", "category_id"), on="film_id", how="inner")
                               .join(category_df.filter(F.col("name")=="Children")
                                                .select("category_id", "name"), on="category_id", how="inner")
                               .select("film_id")
                       )

In [11]:
children_category_group_df = (children_category_df.join(film_actor_df.select("film_id", "actor_id"), on="film_id", how="inner")
                                                  .join(actor_df.select("actor_id", "first_name", "last_name"), on="actor_id", how="inner")
                                                  .groupBy("actor_id", "first_name", "last_name")
                                                  .agg(F.count("film_id").alias("films_count"))
                                                  .select(F.concat_ws(" ", "first_name", "last_name").alias("actor_name"),
                                                          "films_count")                                                  
                             )

In [12]:
#TOP-3 strictly

(children_category_group_df.orderBy(F.col("films_count").desc(), F.col("actor_name").asc())
                           .limit(3)
                           .show())

+-------------+-----------+
|   actor_name|films_count|
+-------------+-----------+
| HELEN VOIGHT|          7|
|KEVIN GARLAND|          5|
|   MARY TANDY|          5|
+-------------+-----------+



In [13]:
#Displaying all actors who share the top positions

windowSpec = Window.orderBy(F.col("films_count").desc())

(children_category_group_df.withColumn("rank", F.rank().over(windowSpec))
                           .filter(F.col("rank") <= 3)
                           .orderBy(F.col("rank").asc(), F.col("actor_name").asc())
                           .show())

26/02/02 22:14:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/02/02 22:14:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/02/02 22:14:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/02/02 22:14:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/02/02 22:14:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/02/02 22:14:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/02/02 2

+-------------+-----------+----+
|   actor_name|films_count|rank|
+-------------+-----------+----+
| HELEN VOIGHT|          7|   1|
|KEVIN GARLAND|          5|   2|
|   MARY TANDY|          5|   2|
|   RALPH CRUZ|          5|   2|
|  WHOOPI HURT|          5|   2|
+-------------+-----------+----+



Stop Spark session:

In [14]:
spark.stop()